In [77]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException, WebDriverException
import time

browser = webdriver.Chrome()
try:
    browser.get('http://dkuhub.dku.edu.cn/psp/CSPRD01/EMPLOYEE/SA/c/SA_LEARNER_SERVICES.CLASS_SEARCH.GBL?Page=SSR_CLSRCH_ENTRY')
except WebDriverException:
    print('Internet Error: Cannot Get dkuhub')
    exit(0)
    
usr = 'ms1129'
pwd = 'qV7urPGc@@YM9Br'
term_name = '2023 Spring Term'

browser.find_element(By.ID, 'expand-netid').click()
WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.ID, 'j_username'))).send_keys(usr)
browser.find_element(By.ID, 'j_password').send_keys(pwd)
browser.find_element(By.ID, 'Submit').click()
browser.switch_to.frame('ptifrmtgtframe')

In [84]:
print({each.text: each.get_attribute('value') for each in Select(browser.find_element(By.ID, 'SSR_CLSRCH_WRK_SUBJECT_SRCH$0')).options})  # subject_dict

{' ': '', 'Arts and Humanities': 'ARHU', 'Behavioral Science': 'BEHAVSCI', 'Biology': 'BIOL', 'Capstone': 'CAPSTONE', 'Chemistry': 'CHEM', 'Chinese': 'CHINESE', 'Computer Science': 'COMPSCI', 'Cultural Anthropology': 'CULANTH', 'Cultures and Movements': 'CULMOVE', 'Economics': 'ECON', 'Electrical & Computer Egr': 'ECE', 'English for Academic Purposes': 'EAP', 'Environment': 'ENVIR', 'Ethics and Leadership': 'ETHLDR', 'French': 'FRENCH', 'German': 'GERMAN', 'Global China Studies': 'GCHINA', 'Global Cultural Studies': 'GCULS', 'Global Health': 'GLHLTH', 'History': 'HIST', 'Information Science': 'INFOSCI', 'Institutions and Governance': 'INSTGOV', 'Italian': 'ITALIAN', 'Japanese': 'JAPANESE', 'Literature': 'LIT', 'Material Science': 'MATSCI', 'Mathematics': 'MATH', 'Media and Arts': 'MEDIART', 'Medical Physics': 'MEDPHY', 'Music': 'MUSIC', 'Neuroscience': 'NEUROSCI', 'Philosophy': 'PHIL', 'Physics': 'PHYS', 'Political Economy': 'POLECON', 'Political Science': 'POLSCI', 'Psychology': 'PSYC

In [82]:
subject_num = len(Select(browser.find_element(By.ID, 'SSR_CLSRCH_WRK_SUBJECT_SRCH$0')).options)
data = {}
t1 = time.time()
term_selector = Select(browser.find_element(By.ID, 'CLASS_SRCH_WRK2_STRM$35$'))
options_text = [each.text for each in term_selector.options]
if term_name not in options_text:
    print('Error: Invalid term, check all terms below\n', '\n'.join(options_text))
term_selector.select_by_visible_text(term_name)
time.sleep(1)
Select(browser.find_element(By.ID, 'SSR_CLSRCH_WRK_ACAD_CAREER$2')).select_by_value('UGRD')
time.sleep(1)
n_frame = 0
for i in range(42, subject_num):
    subject = Select(browser.find_element(By.ID, 'SSR_CLSRCH_WRK_SUBJECT_SRCH$0'))
    if subject.options[i].text == 'Medical Physics' or subject.options[i].text == 'Electrical & Computer Egr':
        continue
    subject.select_by_index(i)
    browser.find_element(By.ID, 'SSR_CLSRCH_WRK_UNITS_MINIMUM$10').send_keys(Keys.ENTER)
    if subject.options[i].text == 'English for Academic Purposes' or subject.options[i].text == 'Chinese':
        browser.switch_to.default_content()
        WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.ID, f'ptModFrame_{n_frame}')))
        browser.switch_to.frame(f'ptModFrame_{n_frame}')
        browser.find_element(By.ID, '#ICSave').click()
        browser.switch_to.default_content()
        browser.switch_to.frame('ptifrmtgtframe')
        n_frame += 1
    WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, 'win0divSSR_CLSRSLT_WRK_GROUPBOX2GP$0')))
    class_num = int(browser.find_element(By.CLASS_NAME, 'PSGROUPBOXLABEL').text.split(' ')[0])
    t = 0
    index = 0
    while t < class_num:
        course_name = WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, f'win0divSSR_CLSRSLT_WRK_GROUPBOX2GP${index}'))).text.strip()
        course = data[course_name] = {}
        classes_num = len(WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, f'ACE_$ICField48${index}'))).find_elements(By.TAG_NAME, 'tr')) // 8
        for j in range(classes_num):
            name, section = WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, f'win0divMTG_CLASSNAME${t}'))).text.split('\n')
            course[name] = {}
            course[name]['section'] = section
            course[name]['number'] = browser.find_element(By.ID, f'win0divMTG_CLASS_NBR${t}').text
            course[name]['time'] = browser.find_element(By.ID, f'win0divMTG_DAYTIME${t}').text.split('\n')
            course[name]['room'] = browser.find_element(By.ID, f'win0divMTG_ROOM${t}').text.split('\n')
            course[name]['instr'] = browser.find_element(By.ID, f'win0divMTG_INSTR${t}').text.replace(',\n', ', ').split('\n')
            course[name]['date'] = browser.find_element(By.ID, f'win0divMTG_TOPIC${t}').text.split('\n')
            browser.execute_script(f"javascript:submitAction_win0(document.win0,'MTG_CLASSNAME${t}');")
            if 'desc' not in course:
                course['desc'] = WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, f'DERIVED_CLSRCH_DESCRLONG'))).text
                course['units'] = browser.find_element(By.ID, f'SSR_CLS_DTL_WRK_UNITS_RANGE').text
                try:
                    course['attr'] = browser.find_element(By.ID, f'SSR_CLS_DTL_WRK_SSR_CRSE_ATTR_LONG').text
                except NoSuchElementException:
                    course['attr'] = ''
                course['mode'] = browser.find_element(By.ID, f'INSTRUCT_MODE_DESCR').text  # online
                course['loc'] = browser.find_element(By.ID, f'CAMPUS_LOC_VW_DESCR').text
                try:
                    course['pre'] = browser.find_element(By.ID, f'SSR_CLS_DTL_WRK_SSR_REQUISITE_LONG').text
                except NoSuchElementException:
                    course['pre'] = ''
            course[name]['cap'] = WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, f'SSR_CLS_DTL_WRK_ENRL_CAP'))).text  # capacity
            course[name]['wait'] = browser.find_element(By.ID, f'win0divSSR_CLS_DTL_WRK_WAIT_CAPlbl').text  # wait list cap
            course[name]['enrl_tot'] = browser.find_element(By.ID, f'SSR_CLS_DTL_WRK_ENRL_TOT').text  # enrollment total
            course[name]['wait_tot'] = browser.find_element(By.ID, f'win0divSSR_CLS_DTL_WRK_WAIT_TOTlbl').text
            course[name]['seats'] = browser.find_element(By.ID, f'SSR_CLS_DTL_WRK_AVAILABLE_SEATS').text
            browser.find_element(By.ID, 'CLASS_SRCH_WRK2_SSR_PB_BACK').click()
            t += 1
        index += 1
    WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, f'CLASS_SRCH_WRK2_SSR_PB_MODIFY'))).click()
    WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, 'SSR_CLSRCH_WRK_SUBJECT_SRCH$0')))
print(time.time() - t1)
print(data)

39.22726488113403
{'WOC 190 - Special Topics in Writing': {'001-SEM': {'section': 'Seven Wk 1', 'number': '1527', 'time': ['MoWe 20:10 - 21:10'], 'room': ['TBA'], 'instr': ['Austin WOERNER'], 'date': ['2023/01/09 - 2023/03/02'], 'cap': '12', 'wait': 'Wait List Capacity', 'enrl_tot': '0', 'wait_tot': 'Wait List Total', 'seats': '12'}, 'desc': 'This is a theme-based writing seminar course through which students strengthen their creative or expository writing skills through exploration of a topic chosen by the instructor. This course is open to all undergraduate students.', 'units': '2 units', 'attr': '', 'mode': 'In Person', 'loc': 'Kunshan Campus', 'pre': '', '002-SEM': {'section': 'Seven Wk 2', 'number': '1528', 'time': ['MoWe 20:10 - 21:10'], 'room': ['TBA'], 'instr': ['Austin WOERNER'], 'date': ['2023/03/20 - 2023/05/11'], 'cap': '12', 'wait': 'Wait List Capacity', 'enrl_tot': '0', 'wait_tot': 'Wait List Total', 'seats': '12'}}, 'WOC 201 - Academic Writing 1: Argumentation Across Dis

In [3]:
term_selector = Select(browser.find_element(By.ID, 'CLASS_SRCH_WRK2_STRM$35$'))
options_text = [each.text for each in term_selector.options]
if term_name not in options_text:
    print('Error: Invalid term, check all terms below\n', '\n'.join(options_text))
term_selector.select_by_visible_text(term_name)
time.sleep(1)
Select(browser.find_element(By.ID, 'SSR_CLSRCH_WRK_ACAD_CAREER$2')).select_by_value('UGRD')
time.sleep(1)
Select(browser.find_element(By.ID, 'SSR_CLSRCH_WRK_LOCATION$6')).select_by_visible_text('Kunshan Campus')
time.sleep(1)
browser.find_element(By.ID, 'SSR_CLSRCH_WRK_UNITS_MINIMUM$10').send_keys(Keys.ENTER)
browser.switch_to.default_content()
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.ID, 'ptModFrame_0')))
browser.switch_to.frame('ptModFrame_0')
browser.find_element(By.ID, '#ICSave').click()
browser.switch_to.default_content()
browser.switch_to.frame('ptifrmtgtframe')
WebDriverWait(browser, 100).until(EC.presence_of_element_located((By.ID, 'win0divSSR_CLSRSLT_WRK_GROUPBOX2GP$0')))
class_num = int(browser.find_element(By.CLASS_NAME, 'PSGROUPBOXLABEL').text.split(' ')[0])
print(class_num)

517


In [33]:
print(browser.find_element(By.ID, f'DERIVED_CLSRCH_DESCRLONG').text)

This course provides a broad overview of the research methods and applications of behavioral science. It introduces core phenomena of human behavior including but not limited to decision making, perception and action, and group interactions. It also considers how behavior is influenced by institutions (e.g., governmental policies, business, environment). The course is expressly interdisciplinary, drawing upon theories and empirical research in psychology, behavioral economics, political science, and other social sciences.


In [5]:
browser.find_element(By.ID, f'DERIVED_CLSRCH_DESCRLONG').text

'Training in close reading and analysis of text remains a foundational skill in the arts and humanities, whether the text is literary or documentary. This core course combines practical training in close reading of a variety of texts, with strategies of analysis that are theoretically informed without, however, offering a comprehensive treatment of theory per se. The course will focus both on reading and analysis of literary texts, and on the nuanced unpacking of documents (official, unofficial, personal) with a view to historical method.'

'4 units'

'30'

'Quantitative Reasoning'

'Prerequisite: COMPSCI 301 or COMPSCI 306 or COMPSCI 308 or COMPSCI 310 or COMPSCI 311'

done


In [ ]:
t = 0
index = 0
data = {}
while t < class_num:
    classes_num = len(browser.find_element(By.ID, f'ACE_$ICField48${index}').find_elements(By.TAG_NAME, 'tr')) // 8
    course_name = browser.find_element(By.ID, f'win0divSSR_CLSRSLT_WRK_GROUPBOX2GP${index}').text.strip()
    course = data[course_name] = {}

    for i in range(classes_num):
        name, section = browser.find_element(By.ID, f'win0divMTG_CLASSNAME${t}').text.split('\n')
        course[name] = {}
        course[name]['section'] = section
        course[name]['number'] = browser.find_element(By.ID, f'win0divMTG_CLASS_NBR${t}').text
        course[name]['time'] = browser.find_element(By.ID, f'win0divMTG_DAYTIME${t}').text.split('\n')
        course[name]['room'] = browser.find_element(By.ID, f'win0divMTG_ROOM${t}').text.split('\n')
        course[name]['instr'] = browser.find_element(By.ID, f'win0divMTG_INSTR${t}').text.replace(',\n', ', ').split('\n')
        course[name]['date'] = browser.find_element(By.ID, f'win0divMTG_TOPIC${t}').text.split('\n')
        t += 1
    index += 1
print(data)